In [ ]:
Overview:
    Tabular time series data with 590k train transactions and 500k test transactions.
443 columns as independent variables
“isFraud” is the dependent variable
Leader Board (LB) is based on the public and private AUC score.
Goal: Predict transactions that are isFraud==1.

Submission: CSV file with TransactionID and Probability of isFraud==1.

Grading: Based on AUC score.
    
    XGBoost is an optimized distributed gradient boosting library designed to have high computation speed & performance.

In [ ]:
Columns explanation
TransactionDT: timedelta from a given reference datetime (not an actual timestamp)
TransactionAMT: transaction payment amount in USD
ProductCD: product code, the product for each transaction
card1 - card6: payment card information, such as card type, card category, issue bank, country, etc.
addr: address
dist: distance
P_ and (R__) emaildomain: purchaser and recipient email domain
C1-C14: counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.
D1-D15: timedelta, such as days between previous transaction, etc.
M1-M9: match, such as names on card and address, etc.
Vxxx: Vesta engineered rich features, including ranking, counting, and other entity relations.
Categorical Features:

ProductCD
card1 - card6
addr1, addr2
P_emaildomain
R_emaildomain
M1 - M9
id_12 - id_38

In [ ]:
#Pseudocode

#import Dependencies
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import confusion_matrix, roc_auc_score, plot_confusion_matrix, precision_score, recall_score, classification_report, plot_roc_curve, roc_curve
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GroupKFold
import gc
import random as r
import joblib

In [ ]:
#import Data

In [ ]:
#show max rows and columns

In [ ]:
#test Train Split

In [ ]:
## delete large unused variables using GC

In [ ]:
# get the metrics in the format for AUC/ Predictions

In [ ]:
#Format Test Predictions

In [ ]:
#Create predictions dataframe

In [ ]:
#build XGB model

In [ ]:
## score prediction

In [ ]:
## test results predictions

In [ ]:
## Getting feature importances

In [ ]:
## setup kfold

In [ ]:
## OOF

In [ ]:
## Getting Feature Importances

In [ ]:
EDA ## collecting all columns

In [ ]:
## Vcol buckets

In [ ]:
## Remove all v columns using below code
tf_V = [bool(re.search("^V"+"[0-9]+",col)) for col in xs.columns.to_list()]
cols = xs.columns[list(~np.array(tf_V))].to_list()
## add specific v columns
cols +=v
print(len(cols))

In [ ]:
## remove time and other columns that don't add to score
cols_rem = ['TransactionID', 'TransactionDT']

cols_rem +=["DayNum","HrOfDay","WkDayNum",'R_emaildomain1', 'R_emaildomain2', 'P_emaildomain1', 'P_emaildomain2','id_31_browser',"DeviceInfo_make"]
cols = set_approach(cols,cols_rem)

In [ ]:
#Clean up D columns

In [ ]:
# One Hot Encoding

In [ ]:
#Adding Dn columns

In [ ]:
#combine and group aggregation (Getting UID)

In [ ]:
#Frequency Encoding

In [ ]:
#Final list of columns

In [ ]:
## Fillna

In [ ]:
#Local validation: Hold out

In [ ]:
## variable to trigger getting FI

In [ ]:
## Linear split 80% 20%

In [ ]:
## Run LV 5 times

In [ ]:
Local Validation: kfold

In [ ]:
## Run 6 time kfold along with OOF prediction and fold based prediction of Test

In [ ]:
Adverserial Validation

In [ ]:
## AV

In [ ]:
if AV:
    df_dom = pd.concat([xs[cols], test_xs[cols]])
    is_test = pd.DataFrame([0]*len(xs) + [1]*len(test_xs))
    idxT,idxV = train_test_split(np.arange(len(df_dom)),test_size=0.2)
    del xs, test_xs; x=gc.collect()

    clf = xgb.XGBClassifier(n_estimators=1000, 
                            max_depth=8, #12
                            learning_rate= 0.05,
                            subsample= 0.6,#0.8 
                            colsample_bytree= 0.4, 
                            random_state = r.randint(0,9999),
                            use_label_encoder=False,                            
    #                         #USE CPU
    #                         nthread=4)
                            # USE GPU
                            tree_method='gpu_hist')

    m = clf.fit(df_dom.iloc[idxT], is_test.iloc[idxT], eval_set=[(df_dom[cols].iloc[idxV],is_test.iloc[idxV])],
                    eval_metric= "auc", verbose=100, early_stopping_rounds=100)

    fi = xgb_fi(m, df_dom,df_dom)
    fi.to_csv("fi_av.csv",index=False)
    fi